## Random assignment, binary CATE example

This is a fully worked-out notebook showing how you would apply auto-causality to a dataset.

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, auto-causality, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import auto_causality
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))


In [2]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%%javascript

// turn off scrollable windows for large output
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
from auto_causality import AutoCausality
from auto_causality.datasets import synth_ihdp
from auto_causality.data_utils import preprocess_dataset

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


### Model fitting & scoring
Here we fit a (selection of) model(s) to the data and score them with the normalized ERUPT metric (chosen to specifically look for differences in impact across customers) on held-out data.

We import an example dataset and pre-process it. The pre-processing fills in the NaNs and one-hot-encodes all categorical and int variables.

If you don't want an int variable to be one-hot-encoded, please cast it to float before preprocessing.

In [4]:
data = synth_ihdp()
treatment = data.treatment
targets = data.outcomes
data_df, features_X, features_W = preprocess_dataset(data.data, treatment, targets)

outcome = targets[0]
train_df, test_df = train_test_split(data_df, test_size=0.2)

In [5]:
print(train_df.head())

     treatment  y_factual        x1        x2        x3        x4        x5  \
277        1.0   4.474727 -0.439977 -0.202946  0.011465 -0.879606 -0.254086   
479        0.0   2.261552  0.532750  0.596582 -1.477987  0.161703  1.058775   
451        1.0   6.417115 -0.548057  0.596582  1.500917 -0.879606 -1.191844   
687        0.0   3.443494  0.921840  0.596582 -0.360898 -0.879606 -1.066810   
514        0.0   2.134798 -1.088461 -1.402238  1.128554  1.203011 -0.441638   

           x6   x7   x8  ...  x17  x18  x19  x20  x21  x22  x23  x24  x25  \
277 -1.023402  1.0  1.0  ...  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
479 -0.857787  1.0  0.0  ...  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
451 -0.857787  1.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
687  0.632754  0.0  0.0  ...  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
514 -0.857787  0.0  0.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   

     random  
277     1.0  
479     1.0  
451     1.0  
687   

Fitting the model is as simple as calling AutoCausality.fit(), with the only required parameter apart from the data being the amount of time you want to give the optimizer, either for the whole run (`time_budget`) or per FLAML component model (`components_time_budget`), or both.

If you want to use specific estimators, comment in the `estimator_list` below to include any estimators whose full name contains any of the elements of `estimator_list`.

The other allowed values are `all` and `auto`, the default is `auto`.


In [ ]:
ac = AutoCausality(
#     time_budget=600,# it's best to specify either time_budget or components_time_budget, and let the other one be inferred
    # estimator_list=[
    #         "Dummy",
    #         "SparseLinearDML",
    #         "ForestDRLearner",
    #         "TransformedOutcome",
    #         "CausalForestDML",
#             ".LinearDML",
#             "DomainAdaptationLearner",
#             "SLearner",
#             "XLearner",
#             "TLearner",
#             "Ortho",
        # ],
    metric="norm_erupt",
    verbose=3,
    components_verbose=2,
    components_time_budget=15,
)


# run autocausality
ac.fit(train_df, treatment, outcome, features_W, features_X)

# return best estimator
print(f"Best estimator: {ac.best_estimator}")
# config of best estimator:
print(f"best config: {ac.best_config}")
# best score:
print(f"best score: {ac.best_score}")


After running a fit, you can resume it without losing past results, for example if you want to search over extra estimators.

In [ ]:
# we can now resume the fit to continue with the init_cfgs which we haven't tried yet
# ac.fit(train_df, treatment, outcome, features_W, features_X,resume=True)
# # return best estimator
# print(f"Best estimator: {ac.best_estimator}")
# # config of best estimator:
# print(f"best config: {ac.best_config}")
# # best score:
# print(f"best score: {ac.best_score}")

In [ ]:
# score all estimators on the test set, which we've kept aside up till now

for est_name, scr in ac.scores.items():
    causal_estimate = scr['estimator']
    scr['scores']['test'] = ac.scorer.make_scores(causal_estimate, test_df, ac.problem, ac.metrics_to_report)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

colors = ([matplotlib.colors.CSS4_COLORS['black']] +
    list(matplotlib.colors.TABLEAU_COLORS) + [
    matplotlib.colors.CSS4_COLORS['lime'],
    matplotlib.colors.CSS4_COLORS['yellow'],
    matplotlib.colors.CSS4_COLORS['pink']
])


plt.figure(figsize = (7,5))
plt.title(outcome)

m1 = "ate"
m2 = "norm_erupt"

for (est, scr), col in zip(ac.scores.items(),colors):
    try:
        sc = [scr["scores"]['train'][m1], scr["scores"]['validation'][m1], scr["scores"]['test'][m1]]
        crv = [scr["scores"]['train'][m2], scr["scores"]['validation'][m2], scr["scores"]['test'][m2]]
        plt.plot(sc, crv, color=col, marker="o", label=est)
        plt.scatter(sc[1:2],crv[1:2], c=col, s=70, label="_nolegend_" )
        plt.scatter(sc[2:],crv[2:], c=col, s=120, label="_nolegend_" )

    except:
        pass
plt.xlabel(m1)
plt.ylabel(m2)

plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)

plt.grid()
plt.show()


In [ ]:
scr = ac.scores[ac.best_estimator]
intrp = scr["scores"]['validation']['intrp']
plt.figure(figsize=(15, 7))
intrp.plot(feature_names=intrp.feature_names, fontsize=10)
plt.title(f"{ac.best_estimator}_{outcome}")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import shap

# and now let's visualize feature importances!
from auto_causality.shap import shap_values

# Shapley values calculation can be slow so let's subsample
this_df = test_df.sample(100)

if "Dummy" not in ac.best_estimator:
    scr = ac.scores[ac.best_estimator]
    print(outcome, ac.best_estimator)
    est = ac.model
    shaps = shap_values(est, this_df)

    plt.title(outcome + '_' + ac.best_estimator.split('.')[-1])
    shap.summary_plot(shaps, this_df[est.estimator._effect_modifier_names])
    plt.show()
else:
    print(f"The best performing model is {ac.best_estimator} which doesn't depend on features")


In [ ]:
# plot out-of sample difference of outcomes between treated and untreated for the points where a model predicts positive vs negative impact
my_est = ac.best_estimator
stats = []

v = ac.scores[my_est]['scores']['test']['values']

sts = ac.scorer.group_ate(test_df.reset_index(), v['norm_policy'])

display(sts)


colors = (matplotlib.colors.CSS4_COLORS['black'],
    matplotlib.colors.CSS4_COLORS['red'],
    matplotlib.colors.CSS4_COLORS['blue'])

grp = sts["policy"].unique()

for i,(p,c) in enumerate(zip(grp, colors)):
    st = sts[sts["policy"] == p]
    plt.errorbar(np.array(range(len(st))) +0.1*i, st["mean"].values[0],  yerr = st["std"].values[0], color=c)
plt.legend(grp)
plt.grid(True)
plt.title(my_est.split('.')[-1])
plt.show()